# Main code

## Libraries and Modules

In [1]:
import glob
import os
import time
import csv

import numpy as np
from scipy.spatial import Voronoi
import cv2

import import_ipynb
import machines
import tas #Threat Assessment Strategy: Voronoi
import pps #Path Planning Strategy: AGA and RHC
import pts #Path Tracking Strategy: Deadlock
import vertices_assembler as assembler
import utils

importing Jupyter notebook from machines.ipynb
importing Jupyter notebook from tas.ipynb
importing Jupyter notebook from node_linker.ipynb
importing Jupyter notebook from pps.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from pts.ipynb
importing Jupyter notebook from vertices_assembler.ipynb


## Global Parameters

In [2]:
parameters = {"deadlock_mov": 0.1,
              "movement"    : 0.1,
              "previous"    : 1,
              "radius"      : 0.1,
              "robots"      : 3,
              "tolerance"   : 0.05
             }

## General Options

* **palette:** Color for each robot.

* **verbose:** Show running data.

* **optimize:** Enable RHC.

* **plot:** Generate images.

* **zoom:** For image dimensions.

* **save_image:** In the machine.

* **save_data:** Only valuable data for analysis.

* **experiments:** Number of experiments for each parameter.

* **seed:** For the generation of pseudo random number.

* **breakpoint:** Number of step to inspect code (data and images). A high number blocks the change of the verbose and plot state. 

* **see_more_steps:** From breakpoint.

* **algorithm:** "aga" or "rhc".

* **path_data:** Path string.

* **path_image:** Path string.

In [6]:
opt = { "algorithm"  : "rhc",
        "breakpoint" : 100,
        "experiments": 1,
        "optimize"   : 1,
        "path_data"  : "/home/leduin/Escritorio/Tesis/Resultados/Datos/",
        "path_image" : "/home/leduin/Escritorio/Tesis/Imagenes/",
        "plot"       : 0,
        "save_image" : 0,
        "save_data"  : 0,
        "see_more_steps" : 0,
        "seed"       : 0,
        "verbose"    : 0,
        "zoom"       : 1,
       }

## Main Loop
`def main_module(key)`

### Parameters:
* **key : string**
    * Name of the parameter

In [4]:
def main_module(key):
    with open(opt["path_data"]+opt["algorithm"]+key+".csv","a") as f:
        if opt["save_data"]:
            csv_writer = csv.writer(f)
        for experiment in range(opt["experiments"]):
            #--------------------------INITIALIZATION--------------------------
            start_time = time.time()
            inst_experiment = experiment + opt["seed"]
            robots = [machines.Robot(i) for i in range(parameters["robots"])]
            # Generate positions, distances, Voronoi attibutes and colors
            utils.pos_gen(parameters["robots"], robots, inst_experiment)
            pos_i = np.array([agent.pos_i for agent in robots])
            pos_f = np.array([agent.pos_f for agent in robots])
            vor_i = Voronoi(pos_i)
            vor_f = Voronoi(pos_f)
            utils.colors(parameters["robots"], robots)
            finished = np.all(pos_i == pos_f) 
            step = 0
            #-------------------COLLISION FREE CONFIGURATION-------------------
            configuration = utils.collision(vor_i.ridge_points,
                                            parameters["radius"], robots)
            if configuration["crash"]:
                print(configuration["message"])
                break
            #-----------------------------MAIN LOOP----------------------------
            while not finished:
                pos_i = np.array([agent.pos_i for agent in robots])
                #Final iteration
                if np.all(pos_i == pos_f):
                    finished = True
                # Generate Voronoi attributes for the current positions
                vor_i = Voronoi(pos_i)
                utils.neighborhood(robots, vor_i.ridge_points)
                if opt["verbose"]:
                    print("==================================================")
                    print('                    MAIN DATA')
                    print("==================================================")
                    print('\nID\tCurrent Pos\tFinal Pos\tColors\tNeighboors')
                    for agent in robots:
                        print('%i\t[%.2F,\t%.2F]\t[%.2F,\t%.2F]\t%s\t'
                              % (agent.id, agent.pos_i[0], agent.pos_i[1],
                                 agent.pos_f[0], agent.pos_f[1], agent.color),
                              agent.neighbors_id)
                for agent in robots:
                    agent.path.append(agent.pos_i)
                # Get the approximated values of inf vertices
                vor_prox = tas.vor_prox(pos_i, opt["verbose"], vor_i)
                # Assemble the vertices and get the ordered vertices
                assembler.main(opt["verbose"], robots, vor_i, vor_prox["edge"])
                # TAS: Generate the BVC
                tas.bvc_gen(parameters["radius"], robots, opt["verbose"],
                            vor_prox)
                # PPS: Get the closer points
                pps.controller(opt, parameters["radius"], robots, step)
                # Plotting
                if opt["plot"]:
                    utils.plot(opt, robots, vor_prox)
                utils.save_image(parameters["robots"], opt, step)
                # PTS: Deadlock testing and solutions
                pts.deadlock_test(opt, parameters, robots, step)
                if opt["verbose"]:
                    print('\nRounds: ',step)
                    print('\n*-*-*-*-*-*-*-*ENDING ROUND*-*-*-*-*-*-*-*-*\n')
                # For activate or deactivate Print Debugging
                step += 1
                if (step == opt["breakpoint"]
                    or (step > opt["breakpoint"] + opt["see_more_steps"])):
                    opt["verbose"] = 1 - opt["verbose"]
                    opt["plot"] = 1 - opt["plot"]
                # UPDATING POSITIONS
                utils.update_pos(parameters["movement"], robots)
            effectiveness = (np.sum([agent.dist_total for agent in robots])
                             / np.sum([agent.dist_travel for agent in robots]))
            if opt["save_data"]:
                csv_writer.writerow([parameters[key], inst_experiment, step,
                                     time.time() - start_time, effectiveness])
            print('Experiment ' + str(inst_experiment) + " done!")

#### Running the code for the number of experiments determined in the dictionary

In [7]:
main_module("robots")

Experiment 0 done!


### One Parameter Analysis

In [7]:
parameter = "robots"
print("Working on",parameter,"...")
previous_value = parameters[parameter]
if parameter == "deadlock_mov":
    parameters[parameter] = 1
elif parameter == "tolerance":
    parameters[parameter] = 0.1
elif parameter == "movement":
    parameters[parameter] = 20
for i in range(2):
    print(parameter,"equal to",parameters[parameter],"...")
    main_module(parameter)
    if parameter == "tolerance":
        parameters[parameter] = round(parameters[parameter]+0.1,2)
    elif parameter == "movement":
        parameters[parameter] += 10 
    else:
        parameters[parameter] += 1
parameters[parameter] = previous_value

Working on robots ...
robots equal to 5 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!
Experiment 16 done!
Experiment 17 done!
Experiment 18 done!
Experiment 19 done!
Experiment 20 done!
Experiment 21 done!
Experiment 22 done!
Experiment 23 done!
Experiment 24 done!
Experiment 25 done!
Experiment 26 done!
Experiment 27 done!
Experiment 28 done!
Experiment 29 done!
robots equal to 6 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!
Experiment 16 done!
Experiment 17 

### All Parameter Analysis

In [8]:
for parameter in parameters:
    print("Working on",parameter,"...")
    previous_value = parameters[parameter]
    if parameter == "deadlock_mov":
        parameters[parameter] = 1
    for i in range(10):
        print(parameter,"equal to",parameters[parameter],"...")
        main_module(parameter)
        parameters[parameter] += 1
    parameters[parameter] = previous_value

Working on robots ...
robots equal to 5 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!


voronoi_constructor.ipynb:16: RuntimeWarning: invalid value encountered in double_scalars
  },


Experiment 16 done!
Experiment 17 done!
Experiment 18 done!
Experiment 19 done!
Experiment 20 done!
Experiment 21 done!
Experiment 22 done!
Experiment 23 done!
Experiment 24 done!
Experiment 25 done!
Experiment 26 done!
Experiment 27 done!
Experiment 28 done!
Experiment 29 done!
robots equal to 6 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!
Experiment 16 done!
Experiment 17 done!
Experiment 18 done!
Experiment 19 done!
Experiment 20 done!
Experiment 21 done!
Experiment 22 done!
Experiment 23 done!
Experiment 24 done!
Experiment 25 done!
Experiment 26 done!
Experiment 27 done!
Experiment 28 done!
Experiment 29 done!
robots equal to 7 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment

KeyboardInterrupt: 

### Generate Video

In [11]:
frameSize = (432, 288)

out = cv2.VideoWriter('/home/leduin/Desktop/Hub/Images/Simple/animation_simple_'
                      +str(robots)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 4,
                      frameSize)

for filename in sorted(glob.glob('/home/leduin/Desktop/Hub/Images/Simple/'
                                 +str(robots)+'/*.png'),
                       key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()